# Location Model

In [1]:

class GpsLocation():
    " location class"
    
    def __init__(self, latitude, longitude):
        " Construct a gps location given a latitude and a longitude"
        
        self.lat = latitude
        self.long = longitude
        
    # getter and setter of latitude
    def _get_lat(self):
        return self._lat
    
    
    def _set_lat(self, lat):
        if isinstance(lat, int) or isinstance(lat, float):
            self._lat = float(lat)
        else:
            raise TypeError(" Latitude must be number !")
            
    lat = property(_get_lat, _set_lat)
    
    
    # getter and setter longitude
    def _get_long(self):
        return self._long
    
    
    def _set_long(self, long):
        if isinstance(long, int) or isinstance(long, float):
            self._long = float(long)
        else:
            raise TypeError(" Longitude must be a number !")
            
    long = property(_get_long, _set_long)
    
    #represent a location in printing
    def __str__(self):
        return f"gps location at latitude = {self.lat}, longitude = {self.long}"
    
    def __repr__(self):
        return f"gps location at latitude = {self.lat}, longitude = {self.long}"

#  Model of a Pair of Locations

In [12]:

class DlaPairLocations():
    """ Distance and Travel-Time class"""
    
    def __init__(self, location1, location2):
        " construit une tondeuse en donnant ses coordonnees et son orientation"
        
        self.loc1 = location1
        self.loc2 = location2
        
    # getter and setter of latitude
    def _get_loc1(self):
        return self._loc1
    
    
    def _set_loc1(self, loc1):
        if isinstance(loc1, GpsLocation):
            self._loc1 = loc1
        else:
            raise TypeError(" Object is not a Location !")
            
    loc1 = property(_get_loc1, _set_loc1)
    
    
    # getter and setter longitude
    def _get_loc2(self):
        return self._loc2
    
    
    def _set_loc2(self, loc2):
        if isinstance(loc2, GpsLocation):
            self._loc2 = loc2
        else:
            raise TypeError(" Object is not a Location !")
            
    loc2 = property(_get_loc2, _set_loc2)
    
    
    # methods
    def dla_route_cal(self, api_keyy):  # douala route calulus
        """ fetches a bicycle distance and time between two points """
        
        #verify if the API is a sting
        if not isinstance(api_keyy, str):
            raise TypeError(" the API key as do be a string !")
        
        
        # importations
        from herepy import GeocoderReverseApi, RoutingApi
        
        # obtain the addresses of the GPS locations
        try:
            geocoder_reverse_api = GeocoderReverseApi(api_key=api_keyy)
            resp_address1 = geocoder_reverse_api.retrieve_addresses(prox=[self.loc1.lat, self.loc1.long], limit=1)
            resp_address2 = geocoder_reverse_api.retrieve_addresses(prox=[self.loc2.lat, self.loc2.long], limit=1)

            # verify if the are in the city of Douala
            if resp_address1.as_dict()['items'][0]['address']['city'].lower() != 'douala':
                raise Exception(f"{self.loc2}, City name not corresponding to Douala !")

            if resp_address2.as_dict()['items'][0]['address']['city'].lower() != 'douala':
                raise Exception(f"{self.loc2}, City name not corresponding to Douala !")

            # fetches a bicycle route between two points 
            routing_api = RoutingApi(api_key=api_keyy)
            resp_route = routing_api.bicycle_route(waypoint_a=[self.loc1.lat, self.loc1.long], 
                                               waypoint_b=[self.loc2.lat, self.loc2.long])

            return {'distance': resp_route.response['route'][0]['summary']['distance'],
               'travel_time': resp_route.response['route'][0]['summary']['travelTime'],
               'base_time': resp_route.response['route'][0]['summary']['baseTime'],}
        except herepy.error.UnauthorizedError:
            print('invalid api key !')
        except herepy.routing_api.NoRouteFoundError:
            print('Route not found !')
        except requests.exceptions.ConnectionError:
            print('You are not connected !')

# Final Program

In [3]:

def final_programme(locat1, locat2, here_api_key):
    from time import sleep
    
    while True:
        pairLocs = DlaPairLocations(location1=locat1, location2=locat2)
        print(pairLocs.dla_route_cal(api_keyy=here_api_key))
    
        # wait for 20 minutes
        sleep(1200)

# Execution

In [ ]:
final_programme(locat1 = GpsLocation(latitude=4.052460, longitude=9.696301),
                locat2 = GpsLocation(latitude=4.056741, longitude=9.721549),
               here_api_key='YFsO5S_Vq13eW2ctRTdUARtOXX0LSiuJtkLvRehruiA') # Akwa and Bepanda locations

{'distance': 4152, 'travel_time': 1239, 'base_time': 1239}


# Tests

In [17]:
import unittest
import herepy
import requests

# GpsLocation class tests
class TestGpsLocations(unittest.TestCase):
    
    def test_input_latitude_longitude_type(self):
        loc = GpsLocation(4.052460,9.696301)
        
        self.assertEqual([loc.lat, loc.long], [4.052460, 9.696301])
        
        with self.assertRaises(TypeError):
            GpsLocation(1,'lion')
        with self.assertRaises(TypeError):
            GpsLocation('lion', 1)     
            

            
# DlaPairLocations test
class TestDlaPairLocations(unittest.TestCase):
    
    def test_input_location_type(self):
        pairLocs = DlaPairLocations(GpsLocation(4.052460,9.696301),GpsLocation(4.056741,9.721549))
    
        self.assertEqual([[pairLocs.loc1.lat, pairLocs.loc1.long], [pairLocs.loc2.lat, pairLocs.loc2.long]],
                          [[4.052460, 9.696301], [4.056741, 9.721549]])
        
        with self.assertRaises(TypeError):
            DlaPairLocations(1,1)
        with self.assertRaises(TypeError):
            DlaPairLocations(1,'lion')
        with self.assertRaises(TypeError):
            DlaPairLocations(GpsLocation(4.052460,9.696301),1)
        with self.assertRaises(TypeError):
            DlaPairLocations('lion', GpsLocation(4.052460,9.696301))
    
    def test_api_key_type(self):
        pairLocs = DlaPairLocations(GpsLocation(4.052460,9.696301),GpsLocation(4.056741,9.721549))
        
        with self.assertRaises(TypeError):
            pairLocs.dla_route_cal(1)
    
    def test_dla_city_locations(self):
        # GpsLocation(-8.916955,13.187748) -> guinea-malabo-talatona GPS location
        pairLocs1 = DlaPairLocations(GpsLocation(4.052460,9.696301),GpsLocation(-8.916955,13.187748))
        pairLocs2 = DlaPairLocations(GpsLocation(-8.916955,13.187748), GpsLocation(4.052460,9.696301))
        
        pairLocs = DlaPairLocations(GpsLocation(4.052460,9.696301),GpsLocation(4.056741,9.721549))
        
        with self.assertRaises(Exception):
            pairLocs1.dla_route_cal(api_keyy='YFsO5S_Vq13eW2ctRTdUARtOXX0LSiuJtkLvRehruiA')
        with self.assertRaises(Exception):
            pairLocs2.dla_route_cal(api_keyy='YFsO5S_Vq13eW2ctRTdUARtOXX0LSiuJtkLvRehruiA')
            
        
unittest.main(argv=[''], verbosity=2, exit=False)

test_api_key_type (__main__.TestDlaPairLocations) ... ok
test_dla_city_location (__main__.TestDlaPairLocations) ... ok
test_input_location_type (__main__.TestDlaPairLocations) ... ok
test_input_latitude_longitude_type (__main__.TestGpsLocations) ... ok

----------------------------------------------------------------------
Ran 4 tests in 6.422s

OK
